# Algoritmos de Machine Learning 

En este documento se puede encontrar la mayoria de algoritmos que se usaron durante la clase de Elements of Machine learning. Estos algoritmos tambien fueron usados en la entrega final del proyecto en donde se realiza la prediccion del winrate de un juego de League of Legends. 

La mayoria de estos algoritmos fueron recolectados de una variedad de laboraorios de la clase de Machine Learning Models y Elements of Machine Learning. Si se necesita referencia adicional sober como aplicarlos, se puede visualizar cada uno de estos algoritmos en el proyecto anteriormente mencionado. 

## Clustering 

### LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Define and fit standardized data
lda = LDA()
ld = lda.fit_transform(wine_features_std, wine_labels)
lda_df = pd.DataFrame(data = ld, 
        columns = ['LDA1', 'LDA2'])
lda_df['Cluster'] = wine_labels

# Print results of classification of training data
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(wine_features_std, wine_labels)))
# print('Accuracy of LDA classifier on test set: {:.2f}'
#      .format(lda.score(X_test, y_test)))

# Print top n observations of LDA df
lda_df.head()

### PCA

In [1]:
from sklearn.decomposition import PCA
# Define number of principal components
# Source: https://cmdlinetips.com/2018/03/pca-example-in-python-with-scikit-learn/

pca = PCA(n_components=3)

# Fit to features
pc = pca.fit_transform(wine_features)

# Dataframe of principal components and wine labels
pc_df = pd.DataFrame(data = pc, 
        columns = ['PC1', 'PC2','PC3'])
pc_df['Cluster'] = wine_labels
pc_df.head()

NameError: name 'PCA' is not defined

### K - Means Clustering 

In [ ]:
from sklearn.cluster import KMeans
cancer_features = cancer_df.loc[:, 'mean radius':]
cancer_labels = cancer_df[['target']]
-------------------------------------------------------
# Define k-means parameters and cluster
clusters = KMeans(n_clusters=2).fit_predict(cancer_features)
-------------------------------------------------------
# Plot the results as a two-dimensional scatter plot
sns.scatterplot(cancer_features['mean radius'], cancer_features['mean smoothness'], hue=clusters) 


### Expectation-maximization clustering


In [ ]:
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse
def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs), )
        
----------------------------------------------------------------------------------
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2, vmin=0, vmax=.2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2).set_ylim(top=.2, bottom = 0, vmin=0, vmax=.2)
        
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)
----------------------------------------------------------------------------------
# EM clustering
gmm = GaussianMixture(n_components=2, covariance_type='full')
gmm.fit(cancer_features[['mean radius', 'mean smoothness']])

plot_gmm(gmm, np.array(cancer_features[['mean radius', 'mean smoothness']]))

### Mean-shift clustering


In [ ]:

from sklearn.cluster import MeanShift, estimate_bandwidth

# Compute clustering with MeanShift
# The following bandwidth can be automatically detected using
bandwidth = estimate_bandwidth(cancer_features, quantile=0.8)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
#ms.fit(cancer_features[['mean radius', 'mean smoothness']])
ms.fit(cancer_features)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)
----------------------------------------------------------------------------------
# Add the centers to a dataframe
centers_df = pd.DataFrame(cluster_centers).head()
# centers_df.columns = ['x', 'y']
centers_df['x'] = centers_df[0]
centers_df['y'] = centers_df[2]
centers_df.head()
----------------------------------------------------------------------------------
# Plot clusters in two dimensions and overlay the centers
sns.scatterplot(x='mean radius', y='mean perimeter', data=cancer_features, hue=labels)
sns.scatterplot(x='x', y='y', data=centers_df, color='green', s=100)

### Hierarchical clustering


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering
from sklearn.cluster import AgglomerativeClustering

for linkage in ('ward', 'average', 'complete', 'single'):
    clustering = AgglomerativeClustering(linkage=linkage, n_clusters=3)
    #t0 = time()
    clustering.fit(cancer_features[['mean radius', 'mean smoothness']])
    #print("%s :\t%.2fs" % (linkage, time() - t0))

    sns.scatterplot(x='mean radius', y='mean smoothness', data=cancer_features, hue=clustering.labels_)
    plt.figure()

### Dendogram Plot 

In [ ]:
# Take top 30 observations and plot dendrogram
linkage_data = linkage(cancer_features[['mean radius', 'mean smoothness']].tail(30), method='centroid')
scipy.cluster.hierarchy.dendrogram(linkage_data)
plt.show()
-------------------------------------------------------------------------
# Show full dataset dendrogram
linkage_data = linkage(cancer_features[['mean radius', 'mean smoothness']], method='centroid')
scipy.cluster.hierarchy.dendrogram(linkage_data)
plt.show()
-------------------------------------------------------------------------
# Now limit the number of trees allowed and re-plot
# Truncate mode = 'lastp': The last p non-singleton clusters formed in the linkage are the only non-leaf nodes in the 
#  linkage; they correspond to rows Z[n-p-2:end] in Z. All other non-singleton clusters are contracted into leaf nodes.
linkage_data = linkage(cancer_features[['mean radius', 'mean smoothness']], method='centroid')
scipy.cluster.hierarchy.dendrogram(linkage_data, truncate_mode='lastp', p=30)
plt.show()
-------------------------------------------------------------------------
# Now limit the number levels in the tree and re-plot
# Truncate mode = 'level': No more than p levels of the dendrogram tree are displayed. A “level” includes all nodes with p
#  merges from the last merge.
linkage_data = linkage(cancer_features[['mean radius', 'mean smoothness']], method='centroid')
scipy.cluster.hierarchy.dendrogram(linkage_data, truncate_mode='level', p=2)
plt.show()

# Regresiones 

### Regresion Logistica

In [ ]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
lr = logreg.fit(thyroid_features,thyroid_labels)
-----------------------------------------------------------------------
import matplotlib.pyplot as plt
x_simple = thyroid_features.iloc[:, 4:]
x_space = np.linspace(-5, 10, 100)
def model(x):
    return 1 / (1 + np.exp(-x))

loss = model(x_space * lr_simple.coef_ + lr_simple.intercept_).ravel()
plt.plot(x_space, loss, color='red', linewidth=3)
plt.scatter(thyroid_features['tsh_diff'].ravel(), thyroid_labels, color='black', zorder=20)

### Regresion Lineal 

In [ ]:
X_train = sm.add_constant(X_train)
simple_model = sm.OLS(y_train,X_train[['firstInhibitor_red', 'const']])
simple_result = simple_model.fit()
-------------------------------------------------------

X_test = sm.add_constant(X_test)
y_pred_simple = simple_result.predict(X_test[['firstInhibitor_red', 'const']])
print(simple_result.summary())


### Regresion Polinomial

In [ ]:
poly_reg = PolynomialFeatures(degree = 3)
X_poly_train = poly_reg.fit_transform(pd.DataFrame(X_train['firstInhibitor_red']))
X_poly_test = poly_reg.fit_transform(pd.DataFrame(X_test['firstInhibitor_red']))
poly_result = poly_reg.fit(X_poly_train, y_train)
poly_model = LinearRegression()
poly_result = poly_model.fit(X_poly_train, y_train)
y_poly_pred = poly_model.predict(X_poly_test)
-----------------------------------------------------------------------------------
poly_model = sm.OLS(y_train,X_poly_train)
poly_result = poly_model.fit()
print(poly_result.summary())


### SVM

In [ ]:
clf = SVC(kernel = 'linear')
clf.fit(iris_featues, iris_labels)

predictions = clf.predict(iris_featues)
--------------------------------------------------------------------------------
results = pd.DataFrame({'Labels':iris_labels, 'Predictions':predictions})
--------------------------------------------------------------------------------
def plotSVC(svc, param, X, y):
    clf = svc
    clf.fit(X, y)

    plt.clf()
    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y, zorder=10, cmap=plt.cm.Paired,
                edgecolor='k', s=20)

    # Circle out the test data
    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=80, facecolors='none',
                zorder=10, edgecolor='k')

    plt.axis('tight')
    x_min = X.iloc[:, 0].min()
    x_max = X.iloc[:, 0].max()
    y_min = X.iloc[:, 1].min()
    y_max = X.iloc[:, 1].max()

    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
    pre_z = svc.predict(np.c_[XX.ravel(), YY.ravel()])

    Z = pre_z.reshape(XX.shape)
    plt.contour(XX, YY, Z, colors=['k', 'k', 'k'],
                linestyles=['--', '-', '--'])
    
    plt.pcolormesh(XX, YY, Z , cmap=plt.cm.Paired)
    plt.title(param)
    plt.show()
--------------------------------------------------------------------------------
kernels = ['linear', 'rbf', 'poly']
for kernel in kernels:
    svc = SVC(kernel=kernel).fit(iris_featues, iris_labels)
    plotSVC(svc, kernel, iris_featues, iris_labels)

# Funciones de Penalizacion 

### Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr = rr.fit(X_train, y_train)
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)
print ("Ridge Regression Train Score low alpha:", Ridge_train_score)
print ("Ridge Regression Test Score low alpha:", Ridge_test_score)


### Lasso Regression

In [ ]:
# Create polynomial regression features of nth degree
poly_reg = PolynomialFeatures(degree = 3)
X_poly_train = poly_reg.fit_transform(pd.DataFrame(X_train_final))
X_poly_test = poly_reg.fit_transform(pd.DataFrame(X_test_final))
poly_result = poly_reg.fit(X_train_final, y_train)

# Fit linear model now polynomial features
poly_model = LinearRegression()
poly_result = poly_model.fit(X_poly_train, y_train)
y_poly_pred = poly_model.predict(X_poly_test)
--------------------------------------------------------------------------------
# Lasso regression model with alpha = 10000 (high penalty) -- now with a lot of new variables
lasso_poly_model_10k = Lasso(alpha=10000, fit_intercept=True)
lasso_poly_model_10k.fit(X_poly_train, y_train)
lasso_poly_10k_pred = lasso_poly_model_10k.predict(X_poly_test)
print(lasso_poly_model_10k.score(X_poly_test, y_test))
lasso_poly_model_10k.coef_
--------------------------------------------------------------------------------
# Plot to compare models
sns.scatterplot(x = X_test_final['city-mpg'], y = y_test.values.ravel())
sns.regplot(x = X_test_final['city-mpg'], y = lr_pred.ravel(), color='g')
sns.regplot(x = X_test_final['city-mpg'], y = y_poly_pred.ravel(), color='gray', order=3)
sns.regplot(x = X_test_final['city-mpg'], y = lasso_poly_10k_pred.ravel(), color='purple', order=3)
--------------------------------------------------------------------------------


# Series de Tiempo

### ARIMA

In [ ]:
from pandas import datetime
from matplotlib import pyplot
from pandas.tools.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot

# Fit AR model
model = ARIMA(y_train, order=(1, 0, 0))
armodel_fit = model.fit(disp=0)
print(armodel_fit.summary())
# plot residual errors
residuals = pd.DataFrame(armodel_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())
--------------------------------------------------------------------------------
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot

# Fit ARIMA model
model = ARIMA(y_train, order=(4, 1, 1))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())
--------------------------------------------------------------------------------
start_index = '2005-04-03 15:00:00'
end_index = '2005-04-04 14:00:00'
arforecast = armodel_fit.predict(start=start_index, end=end_index)
--------------------------------------------------------------------------------
start_index = '2005-04-03 15:00:00'
end_index = '2005-04-04 14:00:00'
forecast = model_fit.predict(start=start_index, end=end_index)
--------------------------------------------------------------------------------
# Plot of original data and predictions
sns.lineplot(x=forecast.index, y=forecast)
sns.lineplot(x=arforecast.index, y=arforecast)
sns.lineplot(x=y_test.index, y=y_test['AH'])
sns.lineplot(x=y_train.tail(24).index, y=y_train['AH'].tail(24))
--------------------------------------------------------------------------------
import statistics as stats
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

forecast_errors = [y_test['AH'][i]-forecast[i] for i in range(len(y_test))]
print('Forecast Errors: %s' % forecast_errors)

print('\nMean Error:')
print(stats.mean(forecast_errors))

print('\nAbsolute Mean Error:')
print(mean_absolute_error(y_test['AH'], forecast))

print('\nRoot Mean Squared Error:')
print(sqrt(mean_squared_error(y_test['AH'], forecast)))
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


# Decision Trees

In [3]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
--------------------------------------------------------------------------------
! pip install graphviz 
# Windows
# !set PATH=PATH;C:\path\to\anaconda\Library\bin\graphviz\

# Mac
# brew install graphviz

# Linux
# sudo apt-get install graphviz
--------------------------------------------------------------------------------
import graphviz 
dot_data = export_graphviz(clf, out_file=None, 
                     feature_names=feature_dummies.columns,  
                     class_names=labels.unique(),  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph
--------------------------------------------------------------------------------
# Hyperparametros
clf = DecisionTreeClassifier(max_depth=2)
clf = clf.fit(X_train, y_train)
dot_data = export_graphviz(clf, out_file=None, 
                     feature_names=feature_dummies.columns,  
                     class_names=labels.unique(),  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph
--------------------------------------------------------------------------------
y_pred = clf.predict(X_test)
--------------------------------------------------------------------------------
plot_confusion_matrix(y_true=y_test, y_pred=y_pred,
                      title='Confusion matrix')

SyntaxError: invalid syntax (<ipython-input-3-86b47dc62aa8>, line 4)

### Arboles de Decision (Con Metodo GridSearch)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
criterion=["gini","entropy"]
max_depth=range(1,20,2)
splitter=["best","random"]
dt=DecisionTreeClassifier()
grid_decision_tree=GridSearchCV(estimator=dt,cv=15,param_grid=dict(criterion=criterion,max_depth=max_depth,splitter=splitter))
--------------------------------------------------------------------------------
dt2=DecisionTreeClassifier(criterion="entropy",max_depth=7, splitter="best")
dt2.fit(X_train,y_train)
print("Score:", dt2.score(X_test,y_test))

### Random Trees

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=20, n_jobs=-1)
clf.fit(X_train, y_train)
--------------------------------------------------------------------------------
clf.estimators_[0:5]
--------------------------------------------------------------------------------
from sklearn.tree import export_graphviz
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
tree_predictions = clf.estimators_[15].predict(X_test)
r2_score(y_test, tree_predictions)
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


### OOB Score 

In [ ]:

clf = RandomForestRegressor(n_estimators=100, 
                            oob_score=True, 
                            n_jobs=-1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
clf.oob_score_, r2_score(y_test, pred)


# Boosting 

### AdaBoost 

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
from sklearn.ensemble import AdaBoostClassifier

# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html
from sklearn.ensemble import AdaBoostRegressor
--------------------------------------------------------------------------------
# Define model parameters
adaboost_reg = AdaBoostRegressor(n_estimators=100, learning_rate=1, loss='linear')
--------------------------------------------------------------------------------
# Train model
adaboost_reg.fit(X_train, y_train)
--------------------------------------------------------------------------------
# Find train and test predictions
prediction_train = adaboost_reg.score(X_train,y_train)
prediction_test = adaboost_reg.score(X_test,y_test)
print(prediction_train)
print(prediction_test)


### XGBoost

In [ ]:
# https://xgboost.readthedocs.io/en/latest/python/python_intro.html
import xgboost as xgb
--------------------------------------------------------------------------------
# Add data to XGBoost data matrices
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
--------------------------------------------------------------------------------
param = {'booster': 'gbtree', 'max_depth': 5, 'eta': 0.8, 'gamma': .01} #Classification objective:binary:logistic, 'objective': 'reg:squarederror' by default 
--------------------------------------------------------------------------------
# Train model
num_round = 100
bst = xgb.train(param, dtrain, num_round)
--------------------------------------------------------------------------------
# Get model predictions
y_train_pred = bst.predict(dtrain)
y_test_pred = bst.predict(dtest)
--------------------------------------------------------------------------------
# Find train and test r^2 values
print(r2_score(y_train, y_train_pred))
print(r2_score(y_test, y_test_pred))


# Neural Networks 

### CNN (Convolusional neural Networks)

In [ ]:
def row_calculator(number_of_images, number_of_columns):
    if number_of_images % number_of_columns != 0:
        return (number_of_images / number_of_columns)+1
    else:
        return (number_of_images / number_of_columns)
    
def display_image(x, img_size, number_of_images):
    plt.figure(figsize = (8, 7))
    if x.shape[0] > 0:
        n_samples = x.shape[0]
        x = x.reshape(n_samples, img_size, img_size)
        number_of_rows = row_calculator(number_of_images, 4)
        for i in range(number_of_images):
            plt.subplot(number_of_rows, 4, i+1)
            plt.imshow(x[i])
            
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(label_names[predicted_label],
                                100*np.max(predictions_array),
                                label_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')
    
--------------------------------------------------------------------------------
data_train = pd.read_csv("csvTrainImages 13440x1024.csv" , header = None).values
data_label = pd.read_csv("csvTrainLabel 13440x1.csv" , header = None)
data_label = data_label.values.astype('int32')-1
data_label = to_categorical(data_label , 28)
data_test = pd.read_csv("csvTestImages 3360x1024.csv" , header = None).values
data_label_test = pd.read_csv("csvTestLabel 3360x1.csv" , header = None)
data_label_test = data_label_test.values.astype('int32')-1
data_label_test = to_categorical(data_label_test , 28)

print(data_train.shape)
print(data_label.shape)
print(data_test.shape)
print(data_label_test.shape)

#normalize 
data_train = data_train/255
data_test = data_test/255
data_train = np.reshape(data_train , (13440 , 32 , 32 , 1))
data_test = np.reshape(data_test , (3360 , 32 , 32 , 1))
input_size = data_train.shape[1:]

--------------------------------------------------------------------------------
cnn_model = keras.Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_size))
cnn_model.add(Conv2D(64 , kernel_size = (5 , 5) , activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128 , activation = 'relu'))
cnn_model.add(Dense(28 , activation = 'softmax'))
--------------------------------------------------------------------------------
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer= 'adam',
              metrics=['accuracy'])
model.summary()
--------------------------------------------------------------------------------
cnn_model_history = model.fit(data_train, data_label,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_data=(data_test, data_label_test))
--------------------------------------------------------------------------------
results = model.evaluate(data_test, data_label_test, verbose=0)
print('Test loss:', results[0])
print('Test accuracy:', results[1]*100)
--------------------------------------------------------------------------------
print(cnn_model_history.history.keys())
# summarize history for accuracy
plt.plot(cnn_model_history.history['accuracy'])
plt.plot(cnn_model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['cnn_train', 'cnn_validation', 'first_train', 'first_validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(cnn_model_history.history['loss'])
plt.plot(cnn_model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['cnn_train', 'cnn_validation', 'first_train', 'first_validation'], loc='upper left')
plt.show()